### Load  Data

In [1]:
import sys
import inspect
sys.path.insert(0, './scripts')

In [2]:
from dataprep import load_dataset, mal_categorizer

In [3]:
train_df = load_dataset('KDDTrain+', 'Field Names', verbose=True)


 ************************ Reading the dataset KDDTrain+ *************************

 It has 125973 rows and 41 columns

 ************************* It has the following columns *************************
Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'is_host_login', 'is_guest_login', 'count',
       'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate',
       'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
       'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
       'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
       'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
       'dst_host_serror_rate', 'dst_host_srv_serror_rate',
       'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'attack

#### Categorize data

labels_5cat: [normal, DOS, R2L, U2R, probing]

labels_2cat: [normal, malicious]

In [4]:
train_df = mal_categorizer(train_df)
train_df[['attack_name', 'labels_5cat', 'labels_2cat']].head(10)

,attack_name,labels_5cat,labels_2cat
0,normal,normal,normal
1,normal,normal,normal
2,neptune,DoS,malicious
3,normal,normal,normal
4,normal,normal,normal
5,neptune,DoS,malicious
6,neptune,DoS,malicious
7,neptune,DoS,malicious
8,neptune,DoS,malicious
9,neptune,DoS,malicious


## Preprocessor skeleton - in progress

In [ ]:
class preprocessor:
    
    def __init__(self, cols_to_filter=None):
        
        self.cols_to_filter = cols_to_filter
    
    def fit(self, X, y=None):
        """learn any information from the training data we may need to transform the test data"""
                
        # > p = preprocessor()
        # > p.fit(X).transform(X)
        
        return self
    
    def transform(self, X, y=None):
        """transform the training or test data"""
        # transform the training or test data based on class attributes learned in the `fit` step
        return X_new

#### Working on feature selection method (Attribute Ratio)

In [ ]:
from collections import OrderedDict
from operator import itemgetter 

def getAR(df, numericCols: list, labelCols: str):
    """TODO documentation: Feature Selection - Attribute Ratio 
    """
    ratio_dict = {}
    
    if numericCols:
        avg_dict = dict(df[numericCols].mean())
        numeric_catCols = numericCols + [labelCols]
        max_cr = dict(df[numeric_catCols].groupby(labelCols).mean().max())
        
        ratio_dict.update({k: max_cr[k]/avg_dict[k] for k in avg_dict.keys()})

    #TODO calculation for binary columns
        
    return OrderedDict(sorted(ratio_dict.items(), key=itemgetter(1), reverse=True))

In [ ]:
getAR(train_feature_df, numeric_cols, 'labels_5cat')